# Introduction to Machine Learning<br><br>Day 03:<br>Text Classification

<center>Dr. William Mattingly<br>
TAP Institute with JSTOR</center>

1) What is text classification?<br>
2) When should I use text classification?<br>
3) spaCy's Text Classification Pipeline<br>
4) Resources for future exploration of ML

## Part One - Introduction to Text Classification

## Part Two - When to Use Text Classification

## Part Three - How to Collate Training Data

## Part Four - Introduction to spaCy's Text Classification Pipeline

## Conclusion

## Resources for Further work with Machine Learning